run GraphcastOperational with 1 GPU and 80 GB memory

Install uv, if necessary. Initialize earth2studio-mpas project ahijevyc/mpas branch and extra graphcast dependency.
Add kernel for Jupyterhub

 ```
curl -LsSf https://astral.sh/uv/install.sh | sh
mkdir earth2studio-mpas
cd earth2studio-mpas
uv init
uv add "earth2studio[graphcast] @ git+https://github.com/ahijevyc/earth2studio.git@ahijevyc/mpas"
uv add metpy
uv add --dev ipykernel ipywidgets
uv run python -m ipykernel install --user --name=earth2studio-mpas --display-name "Python (Earth2Studio MPAS)"
```

In [1]:
import os
from pathlib import Path

import pandas as pd
import xarray as xr
from earth2studio.data.mpas import MPASHybrid
from earth2studio.io import NetCDF4Backend, WPSBackend
from earth2studio.models.px import GraphCastOperational
from earth2studio.run import deterministic

SCRATCH = Path(os.getenv("SCRATCH"))
ai_models_dir = SCRATCH / "ai-models"
date = pd.to_datetime("2018042400", format="%Y%m%d%H")

In [5]:
!nvidia-smi

Mon Nov 24 14:37:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:2B:00.0 Off |                    0 |
| N/A   34C    P0             72W /  700W |      14MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
if True:
    idir = Path(
        "/glade/campaign/mmm/parc/stoedtli/hourly_cycling/6h_cycling/stoedtli_3dhybrid-60-60-iter_O30kmI60km_benchmark_1/CyclingFC"
    )
    mpas_datasrc = MPASHybrid(
        grid_path=idir / f"{date:%Y%m%d%H}/invariant.655362.nc",
        data_path=f"{idir}/%Y%m%d%H/mpasout.%Y-%m-%d_%H.%M.%S.nc",
        pressure_levels=[50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000],
    )

if False:
    idir = Path("/glade/campaign/mmm/parc/schwartz")
    mpas_datasrc = MPASEns(
        grid_path=idir / "MPAS/15-3km_mesh/grid_mesh/x5.6488066.grid_CONUS.nc",
        data_dir=idir / f"HWT{date.year}/mpas",
    )
if False:
    mpas_datasrc = MPASEns(
        grid_path=idir / "MPAS/15km_mesh/grid_mesh/x1.2621442.grid.nc",
        data_dir=idir / f"HWT{date.year}/mpas_15km",
    )

if False:
    idir = Path(
        "/glade/campaign/mmm/parc/stoedtli/hourly_cycling/6h_cycling/stoedtli_3dhybrid-60-60-iter_O30kmI60km_benchmark_1/CyclingFC"
    )
    mpas_datasrc = MPASPres(
        grid_path=idir / f"{date:%Y%m%d%H}/invariant.655362.nc",
        data_path=[
            f"{idir}/%Y%m%d%H/diag.%Y-%m-%d_%H.%M.%S.nc",
            f"{idir}/%Y%m%d%H/p_sfc.%Y-%m-%d_%H.%M.%S.nc",
        ],
    )

2025-11-24 14:48:40.054 | INFO     | earth2studio.data.mpas:_prepare_regridding_indices:104 - Loading cached regridding indices from /glade/u/home/ahijevyc/.cache/earth2studio/mpas_hybrid/invariant.655362_0.25x0.25.npz


In [ ]:
model_class = GraphCastOperational
model = model_class.load_model(model_class.load_default_package())

In [6]:
ds = mpas_datasrc(
    ["20180423T18"],
    ["z1000", "z500", "z", "lsm"],
)
ds

2025-11-24 11:37:51.460 | INFO     | earth2studio.data.mpas:_load_and_process:289 - Requesting source variables for time 20180423T18: ['pressure_p', 'zgrid', 'pressure_base', 'surface_pressure', 'ter', 't2m', 'landmask']
2025-11-24 11:37:51.555 | INFO     | earth2studio.lexicon.mpas:xtime:554 - Decoding 'initial_time' variable.
2025-11-24 11:37:53.781 | INFO     | earth2studio.lexicon.mpas:xtime:573 - Decoding 'xtime' (valid time) variable.
2025-11-24 11:37:56.772 | INFO     | earth2studio.data.mpas:_load_and_process:324 - 3D variable requested. Performing vertical interpolation to [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000] hPa.
2025-11-24 11:37:56.773 | INFO     | earth2studio.data.mpas:_interpolate_to_pressure_levels:400 - Interpolating variable: geopotential
2025-11-24 11:37:56.775 | INFO     | earth2studio.data.mpas:_interpolate_to_pressure_levels:417 - Using ln(p) interpolation for geopotential
2025-11-24 11:37:59.833 | INFO     | earth2studio.data.mpas:_loa

<xarray.DataArray (time: 1, variable: 4, lat: 721, lon: 1440)> Size: 33MB
array([[[[2.72639082e+04, 2.72639082e+04, 2.72639082e+04, ...,
          2.72639082e+04, 2.72639082e+04, 2.72639082e+04],
         [2.72836914e+04, 2.72836914e+04, 2.72836914e+04, ...,
          2.72836914e+04, 2.72836914e+04, 2.72836914e+04],
         [2.71883965e+04, 2.71883965e+04, 2.71883965e+04, ...,
          2.71883965e+04, 2.71883965e+04, 2.71883965e+04],
         ...,
         [8.80882446e+02, 8.80882446e+02, 8.80882446e+02, ...,
          8.80882446e+02, 8.80882446e+02, 8.80882446e+02],
         [8.72573059e+02, 8.72573059e+02, 8.72573059e+02, ...,
          8.72573059e+02, 8.72573059e+02, 8.72573059e+02],
         [8.72613220e+02, 8.72613220e+02, 8.72613220e+02, ...,
          8.72613220e+02, 8.72613220e+02, 8.72613220e+02]],

        [[4.83016641e+04, 4.83016641e+04, 4.83016641e+04, ...,
          4.83016641e+04, 4.83016641e+04, 4.83016641e+04],
         [4.82990781e+04, 4.82990781e+04, 4.82990781e+04, ...,
          4.82990781e+04, 4.82990781e+04, 4.82990781e+04],
         [4.82984922e+04, 4.82984922e+04, 4.82984922e+04, ...,
          4.82984922e+04, 4.82984922e+04, 4.82984922e+04],
...
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

        [[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]]],
      shape=(1, 4, 721, 1440))
Coordinates:
  * lat           (lat) float64 6kB -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
  * lon           (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    initial_time  datetime64[ns] 8B 2018-04-14T18:00:00
    nCells        (lat, lon) int64 8MB 8 8 8 8 8 8 8 8 8 8 ... 2 2 2 2 2 2 2 2 2
    nVertLevels   int64 8B 0
  * variable      (variable) object 32B 'z1000' 'z500' 'z' 'lsm'
  * time          (time) <U11 44B '20180423T18'

In [3]:
ds.sel(variable="msl").plot(col="time", robust=False, cmap="nipy_spectral")

NameError: name 'ds' is not defined

In [8]:
model.input_coords()["variable"]

array(['t2m', 'msl', 'u10m', 'v10m', 'tp06', 't50', 't100', 't150',
       't200', 't250', 't300', 't400', 't500', 't600', 't700', 't850',
       't925', 't1000', 'z50', 'z100', 'z150', 'z200', 'z250', 'z300',
       'z400', 'z500', 'z600', 'z700', 'z850', 'z925', 'z1000', 'u50',
       'u100', 'u150', 'u200', 'u250', 'u300', 'u400', 'u500', 'u600',
       'u700', 'u850', 'u925', 'u1000', 'v50', 'v100', 'v150', 'v200',
       'v250', 'v300', 'v400', 'v500', 'v600', 'v700', 'v850', 'v925',
       'v1000', 'w50', 'w100', 'w150', 'w200', 'w250', 'w300', 'w400',
       'w500', 'w600', 'w700', 'w850', 'w925', 'w1000', 'q50', 'q100',
       'q150', 'q200', 'q250', 'q300', 'q400', 'q500', 'q600', 'q700',
       'q850', 'q925', 'q1000', 'z', 'lsm'], dtype='<U5')

In [9]:
model_abbrev = model.__class__.__name__.upper()[0:4]

nsteps = 1
ofile = SCRATCH / f"tmp/test_{model_abbrev}:{date:%Y-%m-%d_%H}.nc"
if os.path.exists(ofile):
    os.remove(ofile)
io = WPSBackend(SCRATCH / "tmp", model_source=model.__class__.__name__, static_fields=["lsm", "z"])
# io = NetCDF4Backend(ofile, backend_kwargs={"mode": "w"})
io = deterministic([date], nsteps, model, mpas_datasrc, io)
io.close()
print(ofile)

2025-11-24 11:38:21.221 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-11-24 11:38:21.221 | INFO     | earth2studio.run:deterministic:82 - Inference device: cuda
2025-11-24 11:38:21.222 | INFO     | earth2studio.data.mpas:_load_and_process:289 - Requesting source variables for time 2018-04-23T18:00:00.000000000: ['uReconstructZonal', 'pressure_p', 'zgrid', 'uReconstructMeridional', 'pressure_base', 'surface_pressure', 'ter', 'rainc', 'u10', 't2m', 'w', 'theta', 'landmask', 'rainnc', 'v10', 'qv']
2025-11-24 11:38:21.322 | INFO     | earth2studio.lexicon.mpas:xtime:554 - Decoding 'initial_time' variable.
2025-11-24 11:38:23.811 | INFO     | earth2studio.lexicon.mpas:xtime:573 - Decoding 'xtime' (valid time) variable.
2025-11-24 11:38:27.266 | INFO     | earth2studio.lexicon.mpas:derive_variables:435 - Deriving pressure vertical velocity from geometric w.
2025-11-24 11:38:28.086 | INFO     | earth2studio.lexicon.mpas:derive_variables:482 - Deriving mslp fro


Running inference:   0%|          | 0/2 [00:00<?, ?it/s]
                                                     
Running inference:  50%|█████     | 1/2 [00:08<00:08,  8.56s/it]

2025-11-24 11:39:21.014 | INFO     | earth2studio.io.wps:write:101 - Capturing static fields from initial time step: ['lsm', 'z']


2025-11-24 11:39:58.363002: W external/xla/xla/service/gpu/autotuning/gemm_fusion_autotuner.cc:1092] Compiling 551 configs for 17 fusions on a single thread.

Running inference: 100%|██████████| 2/2 [02:35<00:00, 78.00s/it]


2025-11-24 11:41:48.559 | SUCCESS  | earth2studio.run:deterministic:146 - Inference complete
2025-11-24 11:41:48.599 | INFO     | earth2studio.io.wps:close:228 - Overriding variables with stored static fields.
2025-11-24 11:41:48.600 | INFO     | earth2studio.io.wps:close:242 - Writing final forecast step for time 2018-04-24_00:00:00 (F006) to /glade/derecho/scratch/ahijevyc/tmp/EART:2018-04-24_00
2025-11-24 11:41:48.930 | INFO     | earth2studio.io.wps:close:304 - WPSBackend closed. Final output: /glade/derecho/scratch/ahijevyc/tmp/EART:2018-04-24_00
/glade/derecho/scratch/ahijevyc/tmp/test_GRAP:2018-04-24_00.nc


In [16]:
dir(io)

['HEADER_FORMAT',
 'PROJECTION_FORMAT_LATLON',
 'VARIABLE_MAP',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__non_callable_proto_members__',
 '__parameters__',
 '__protocol_attrs__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_is_protocol',
 '_is_runtime_protocol',
 '_write_complete_field',
 '_write_record',
 'add_array',
 'close',
 'final_data_package',
 'model_source',
 'output_dir',
 'static_field_names',
 'stored_static_data',
 'write']

# That was Graphcast. Now try Pangu

In [6]:
import plot_ensemble
from earyth2studio.models import Pangu6
from metpy.constants import Re
from s3_run_pangu_ecmwf import run_inference, setup_model_sessions

# date = pd.to_datetime("2024042400", format="%Y%m%d%H")
ic = "mpas"
fhr_end = 120

<Quantity(6371008.77, 'meter')>

In [7]:
ort_session_24, ort_session_6 = setup_model_sessions(ai_models_dir)

In [9]:
output_dir = Path(f"/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather/{date:%Y%m%d%H}")

da = ds.rename(variable="channel")
inferences = run_inference(da, ort_session_24, ort_session_6, fhr_end)
for fcst in inferences:
    fhr = fcst.prediction_timedelta.squeeze() / pd.to_timedelta("1h")
    output_filename = os.path.join(output_dir, f"pangu_{ic}_hybrid_pred_{fhr:03.0f}.nc")
    print(output_filename)
    # zlib compression can reduce by 1/3 but takes long time (versus almost instantaneous)
    fcst.to_netcdf(output_filename)

NameError: name 'ds' is not defined

In [ ]:
output_dir = Path(f"/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather/{date:%Y%m%d%H}")

for member in ds.member.data:
    da = ds.sel(member=member).rename(variable="channel").rename(Time="time")
    all_exist = True
    for fhr in range(6, fhr_end + 1, 6):
        output_filename = os.path.join(output_dir, f"pangu_{ic}{member}_pred_{fhr:03.0f}.nc")
        if not os.path.exists(output_filename):
            print(output_filename, "no exists")
            all_exist = False
            break
    if all_exist:
        print(f"all {date} {ic}{member} exist")
        continue
    inferences = run_inference(da, ort_session_24, ort_session_6, fhr_end)
    for fcst in inferences:
        fhr = fcst.prediction_timedelta.squeeze() / pd.to_timedelta("1h")
        output_filename = os.path.join(output_dir, f"pangu_{ic}{member}_pred_{fhr:03.0f}.nc")
        print(output_filename)
        # zlib compression can reduce by 1/3 but takes long time (versus almost instantaneous)
        fcst.to_netcdf(output_filename)

In [ ]:
ifiles = []
for member in ds.member.data:
    ifiles.append(sorted(list(output_dir.glob(f"pangu_{ic}{member}_pred_???.nc"))))
da = (
    xr.open_mfdataset(
        ifiles,
        combine="nested",
        concat_dim=["member", "prediction_timedelta"],
    )
    .rename(lat="latitude", lon="longitude")
    .assign_coords(member=ds.member)
    .sel(channel="z500")
    .rename(__xarray_dataarray_variable__="z", prediction_timedelta="step")
    .squeeze(dim="init_time")
)
da

In [ ]:
import argparse

args = argparse.Namespace
args.ic = ic.upper()
args.model = "panguweather"
fig = plot_ensemble.plot_forecast_grid(
    args, da.sortby("member"), plotdays=range(1, fhr_end // 24 + 1)
)

In [ ]:
member = 4
inferences = xr.open_mfdataset(output_dir.glob(f"pangu_{ic}{member}_pred_???.nc"))
inferences = inferences.sel(lat=slice(60, 20), lon=slice(220, 300))
inferences.sel(channel="z500").squeeze().__xarray_dataarray_variable__.plot(
    col="prediction_timedelta", col_wrap=6
)

In [ ]:
inferences